In [25]:
import pandas as pd

pd.set_option('display.max_columns', None)  # None means unlimited
pd.set_option('display.max_rows', None)     # None means unlimited
pd.set_option('display.max_colwidth', None) # None means show entire content of the column

In [26]:
class Config:
    def __init__(self):
        self.az_storage_conn_str = "DefaultEndpointsProtocol=https;AccountName=estateadviserstorage;AccountKey=Y52EdpNysG+MJetBBg7T+JeLfC/H8ZkB0HyGdRG+NItsVcY5KsKINikApihU4OqgERa2frz1gCVw+AStUiwuzg==;EndpointSuffix=core.windows.net"
        self.az_storage_container_name = "models"
        self.yrs_to_predict = 5
        self.hist_start_year = 2015
        self.hist_batch_incr_days = 90
        self.active_listing_days = 30
        
config = Config()

In [41]:


import pandas as pd


# Configure your connection string appropriately


In [45]:
from datetime import datetime, timezone

# Get the current UTC date and time
utc_now = 

# If you only need the date part
utc_date = utc_now

print(type(utc_date))


<class 'datetime.date'>


In [42]:
import geopy.distance
import pandas as pd
from geopy.geocoders import Nominatim
import pickle
import time
import uuid

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from homeharvest import scrape_property
from datetime import datetime, timedelta
from azure.storage.blob import BlobServiceClient, BlobBlock



        


class PropertyDatasetProcessor:
    def __init__(self, df, city):
        self.dataset = df
        self.city = city

        geolocator = Nominatim(user_agent="RealEstateAdvisor")
        location = geolocator.geocode(f"Downtown {city}")
        self.downtown_lat, self.downtown_lon = location.latitude, location.longitude

    @staticmethod
    def calc_lat_lon_dist(lat1, lon1, lat2, lon2):
        if pd.isna(lat1) or pd.isna(lon1) or pd.isna(lat2) or pd.isna(lon2):
            return None
        return round(geopy.distance.geodesic((lat1, lon1), (lat2, lon2)).km, 1)

    @staticmethod
    def calc_baths_num(full_baths, half_baths):
        if pd.isna(full_baths) and pd.isna(half_baths):
            return 0.0
        elif pd.isna(full_baths):
            return half_baths * 0.5
        elif pd.isna(half_baths):
            return full_baths
        else:
            return full_baths + 0.5 * half_baths

    def clean_dataset(self):
        dataset = self.dataset[(self.dataset.city == self.city)]

        dataset["distance_to_downtown"] = dataset.apply(
            lambda row: self.calc_lat_lon_dist(
                row["latitude"], row["longitude"], self.downtown_lat, self.downtown_lon
            ),
            axis=1,
        )
        dataset["baths"] = dataset.apply(
            lambda row: self.calc_baths_num(row["full_baths"], row["half_baths"]),
            axis=1,
        )
        dataset["sqft"] = dataset.apply(
            lambda row: (
                0.0 if pd.isna(row["sqft"]) and row["style"] == "LAND" else row["sqft"]
            ),
            axis=1,
        )
        dataset["style"] = dataset.apply(
            lambda row: "OTHER" if pd.isna(row["style"]) else row["style"], axis=1
        )
        dataset["lot_sqft"] = dataset.apply(
            lambda row: 0.0 if pd.isna(row["lot_sqft"]) else row["lot_sqft"], axis=1
        )
        dataset["hoa_fee"] = dataset.apply(
            lambda row: 0.0 if pd.isna(row["hoa_fee"]) else row["hoa_fee"], axis=1
        )
        dataset["stories"] = dataset.apply(
            lambda row: 0.0 if pd.isna(row["stories"]) else row["stories"], axis=1
        )
        dataset["beds"] = dataset.apply(
            lambda row: 0.0 if pd.isna(row["beds"]) else row["beds"], axis=1
        )
        dataset["sold_year"] = pd.to_datetime(dataset["last_sold_date"]).apply(
            lambda x: x.year
        )

        dataset.dropna(
            subset=["year_built", "sqft", "distance_to_downtown", "parking_garage", "sold_year"],
            inplace=True,
        )
        dataset["age"] = dataset.apply(
            lambda row: row["sold_year"] - row["year_built"], axis=1
        )

        return dataset


def train_model(dataset):
    cols_to_drop = [
        "property_url",
        "status",
        "street",
        "unit",
        "city",
        "state",
        "days_on_mls",
        "list_price",
        "list_date",
        "latitude",
        "longitude",
        "primary_photo",
        "mls",
        "mls_id",
        "price_per_sqft",
        "alt_photos",
        "style",
        "full_baths",
        "half_baths",
        "last_sold_date",
        "sold_price",
    ]

    X = dataset.drop(cols_to_drop, axis=1)
    y = dataset["sold_price"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )

    rf_model = RandomForestRegressor(n_estimators=50, random_state=42)
    rf_model.fit(X_train, y_train)
    model_score = rf_model.score(X_test, y_test)
    print(f"Model trained with the score: {model_score}")

    return rf_model


def get_chunk_blocks(data, blob_client, chunk_size=4 * 1024 * 1024):
    block_list = []

    index = 0
    while index < len(data):
        chunk_data = data[index : index + chunk_size]

        if not chunk_data:
            break
        blk_id = str(uuid.uuid4())
        blob_client.stage_block(block_id=blk_id, data=chunk_data)
        block_list.append(BlobBlock(block_id=blk_id))

        index += chunk_size

    return block_list


def write_model_to_storage(model, city: str, state: str):
    serialized_model = pickle.dumps(model)

    blob_service_client = BlobServiceClient.from_connection_string(
        config.az_storage_conn_str
    )
    blob_name = f"{city.lower()}_{state.lower()}.pkl"
    blob_client = blob_service_client.get_blob_client(
        container=config.az_storage_container_name, blob=blob_name
    )

    block_list = get_chunk_blocks(
        serialized_model, blob_client, chunk_size=4 * 1024 * 1024
    )
    blob_client.commit_block_list(block_list)

    print(f"{blob_name} uploaded successfully.")

In [ ]:
location, city, state = "Miami, FL", "Miami", "FL"

scrape_historical_sales(location, city, state, engine)

2024-04-22 15:51:10,615 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2024-04-22 15:51:10,615 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-22 15:51:10,705 INFO sqlalchemy.engine.Engine SELECT schema_name()
2024-04-22 15:51:10,706 INFO sqlalchemy.engine.Engine [generated in 0.00178s] ()
2024-04-22 15:51:10,914 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2024-04-22 15:51:10,915 INFO sqlalchemy.engine.Engine [generated in 0.00162s] ()
2024-04-22 15:51:11,004 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2024-04-22 15:51:11,004 INFO sqlalchemy.engine.Engine [generated in 0.00177s] ()
2024-04-22 15:51:11,204 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-22 15:51:11,206 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLE

In [ ]:
df.info()

In [11]:
df['list_date']

0         2015-03-25
1         2015-01-10
2         2014-10-30
3         2015-03-18
4         2014-11-13
5         2015-02-21
6         2014-04-29
7               None
8         2015-03-21
9         2015-03-08
10        2014-11-10
11        2015-03-15
12        2015-03-25
13        2015-03-25
14        2015-02-14
15        2015-02-03
16        2015-03-14
17        2015-02-12
18        2014-11-22
19              None
20        2014-12-23
21        2014-09-28
22        2015-03-03
23        2015-03-14
24        2015-03-10
25        2014-07-12
26        2014-12-14
27        2015-01-28
28        2015-02-28
29        2015-03-30
30        2015-03-17
31        2015-02-28
32        2014-12-03
33        2015-01-01
34        2014-12-07
35              None
36              None
37        2014-11-15
38        2015-03-16
39        2015-02-14
40              None
41        2015-01-25
42        2015-05-17
43              None
44        2015-03-22
45        2014-10-02
46        2015-01-06
47        201

In [20]:
len(sql_df)

0

AttributeError: 'DataFrame' object has no attribute 'to_datetime'

In [13]:
test_df = scrape_property(
                location="Seattle, WA",
                listing_type="sold",  # or (for_sale, for_rent)
                date_from="2024-01-01",
                date_to="2024-01-05",
            )

In [16]:
test_df['last_sold_date'] = pd.to_datetime(test_df['last_sold_date'], errors='coerce')

test_df['last_sold_date'].max()

Timestamp('2024-01-05 00:00:00')